In [1]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

/opt/anaconda3/envs/atlas/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
367,Probably the most accurate Stephen King adapti...,positive
163,"When I was a kid, I remember watching this whi...",negative
600,This movie is... horrible and wonderful at the...,negative
570,This is definitely one of the weaker of the se...,negative
888,I was dragged to this movie about four years a...,positive


In [4]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [5]:
df = normalize_text(df)
df.head()

,review,sentiment
367,probably accurate stephen king adaption yet su...,positive
163,kid remember watching visiting friend uncle ph...,negative
600,movie is horrible wonderful time first saw yea...,negative
570,definitely one weaker series carry film lack u...,negative
888,dragged movie four year ago french actress fri...,positive


In [6]:
df['sentiment'].value_counts()

sentiment
negative    267
positive    233
Name: count, dtype: int64

In [7]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [8]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
367,probably accurate stephen king adaption yet su...,1
163,kid remember watching visiting friend uncle ph...,0
600,movie is horrible wonderful time first saw yea...,0
570,definitely one weaker series carry film lack u...,0
888,dragged movie four year ago french actress fri...,1


In [9]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [15]:
vectorizer = CountVectorizer(max_features=100)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [17]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/vedmukherjee15/Capstone-Project.mlflow')
dagshub.init(repo_owner='vedmukherjee15', repo_name='Capstone-Project', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")

2026-02-14 13:10:55,399 - INFO - HTTP Request: GET https://dagshub.com/api/v1/repos/vedmukherjee15/Capstone-Project "HTTP/1.1 200 OK"


Initialized MLflow to track repo "vedmukherjee15/Capstone-Project"

2026-02-14 13:10:55,404 - INFO - Initialized MLflow to track repo "vedmukherjee15/Capstone-Project"


Repository vedmukherjee15/Capstone-Project initialized!

2026-02-14 13:10:55,404 - INFO - Repository vedmukherjee15/Capstone-Project initialized!


<Experiment: artifact_location='mlflow-artifacts:/28f4292e0b5b4d1399acb6c6e5d7c0eb', creation_time=1771054149854, experiment_id='0', last_update_time=1771054149854, lifecycle_stage='active', name='Logistic Regression Baseline', tags={'mlflow.experimentKind': 'custom_model_development'}>

In [18]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 100)
        mlflow.log_param("test_size", 0.25)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)

2026-02-14 13:11:01,652 - INFO - Starting MLflow run...
2026-02-14 13:11:02,110 - INFO - Logging preprocessing parameters...
2026-02-14 13:11:04,298 - INFO - Initializing Logistic Regression model...
2026-02-14 13:11:04,300 - INFO - Fitting the model...
2026-02-14 13:11:04,323 - INFO - Model training complete.
2026-02-14 13:11:04,324 - INFO - Logging model parameters...
2026-02-14 13:11:04,644 - INFO - Making predictions...
2026-02-14 13:11:04,647 - INFO - Calculating evaluation metrics...
2026-02-14 13:11:04,657 - INFO - Logging evaluation metrics...
2026-02-14 13:11:05,981 - INFO - Saving and logging the model...
2026/02/14 13:11:05 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/opt/anaconda3/envs/atlas/lib/python3.10/site-packages/mlflow/models/model.py:1209: FutureWarning: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can

🏃 View run orderly-hog-999 at: https://dagshub.com/vedmukherjee15/Capstone-Project.mlflow/#/experiments/0/runs/60fb9442e47e4e29b7abc6a9a7ec2d87
🧪 View experiment at: https://dagshub.com/vedmukherjee15/Capstone-Project.mlflow/#/experiments/0
